# ライントレース+物体検知

In [1]:
import sys
sys.path.append( "/home/jetbot/g031r066/darknet" )
print(sys.path)

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/jetbot/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/local/lib/python3.6/dist-packages/torchvision-0.4.0a0+d31eafa-py3.6-linux-aarch64.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_SSD1306-1.6.2-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_MotorHAT-1.4.0-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_GPIO-1.0.4-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/spidev-3.4-py3.6-linux-aarch64.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_PureIO-1.0.4-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg', '/usr/lib/python3/dist-packages', '/usr/lib/python3.6/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/home/jetbot/.ipython', '/home/jetbot/g031r066/darknet']


## 物体検知側の下準備

In [2]:
import darknet
import darknet_images
import time
"""
load model description and weights from config files
args:
    config_file (str): path to .cfg model file
    data_file (str): path to .data model file
    weights (str): path to weights
returns:
    network: trained model
    class_names
    class_colors
"""
"""
network, class_names, class_colors = darknet.load_network(
    '/home/jetbot/g031r066/darknet/cfg/yolov4-tiny.cfg',  
    '/home/jetbot/g031r066/darknet/cfg/coco.data', 
    '/home/jetbot/g031r066/darknet/weights/yolov4-tiny.weights'
    )


"""
network, class_names, class_colors = darknet.load_network(
        '/home/jetbot/g031r066/darknet/custom/yolov4-tiny-custom.cfg', 
        '/home/jetbot/g031r066/darknet/custom/custom.data',  
        '/home/jetbot/g031r066/darknet/custom/backup/yolov4-tiny-custom_final.weights'
    )

## ライントレース側の下準備

In [3]:
# インポート
import torchvision
import torch

# 自分で作成したライントレースのモデルを読み込む
model_rf = torchvision.models.resnet18(pretrained=False)
model_rf.fc = torch.nn.Linear(512, 2)
model_rf.load_state_dict(torch.load('/home/jetbot/g031r066/note_nvidia/road_following/best_steering_models/[nvidia_circuit-ver1.2]_res18.pth'))
# GPU側に転送
device = torch.device('cuda')
model_rf = model_rf.to(device)
model_rf = model_rf.eval().half()

In [4]:
# カメラから読み込む映像への前処理を実装
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
# 諸々インポート
from IPython.display import display
import ipywidgets
import traitlets

# 駆動系周りの準備
from jetbot import Robot

robot = Robot()  # 駆動系を制御できるモジュールのインポート

# 制御するバーの設定
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
# 描画
#display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# 自Jetbotにおいては、以下の設定だと良い感じに動く
# speed gain = 0.1 2022/09/09 0.15の方が綺麗
# steering gain = 0.05
# steering kd = 0
# steering gain = 0

In [6]:
# 現在の動作を示すバーを描画
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

#display(ipywidgets.HBox([y_slider, speed_slider]))
#display(x_slider, steering_slider)

## 本処理

In [7]:
from jetbot import Camera, bgr8_to_jpeg
import datetime
import time
import queue

# カメラ起動
camera = Camera(width=300, height=300, capture_width=1280, capture_height=720)  # デフォルトでは Camera(width=224, height=224, fps=21, capture_width=3280, capture_height=2464)
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# メインスレッド<->サブスレッド で画像をやり取りするためのキューを作っておく
queue_rf = queue.Queue()  # road_following()用
queue_od = queue.Queue()  # object_detection()用

In [8]:
blocked_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
bbox_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
label_widget = ipywidgets.IntText(value=1, description='tracked label')
speed_widget = ipywidgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = ipywidgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')
'''
display(ipywidgets.VBox([
    ipywidgets.HBox([image_widget, blocked_widget),
    label_widget,
    speed_widget,
    turn_gain_widget
]))
'''
width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

angle = 0.0
angle_last = 0.0

# ライントレースを実装する関数
def road_following(snapshot):
    global angle, angle_last

    xy = model_rf(preprocess(snapshot)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    left_motor_rf = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    right_motor_rf = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

    queue_rf.put(left_motor_rf)
    queue_rf.put(right_motor_rf)

# 物体検知を実装する関数
def object_detection(snapshot):
    
    re = 0
    prev_time = time.time()
    thresh = .45
    image, detections = darknet_images.image_detection(snapshot, network, class_names, class_colors, thresh)
    darknet.print_detections(detections, True)  # True: 各物体の座標・幅・高さを表示
    fps = int(1/(time.time() - prev_time))
    print("FPS: {}".format(fps), flush=True)
    print("Predicted in {} seconds".format((time.time() - prev_time)), flush=True)
    
    label_list=[]
    confidence_list=[]
    bbox_list=[]
    for label, confidence, bbox in detections: 
        label_list.append(label)
        confidence_list.append(confidence)
        bbox_list.append(bbox)
    
    queue_od.put(image)  # メインスレッドにbboxを描画した画像を送る
    
    if("Traffic light" in label_list):  # 信号機部分を切り出し
        # 対象の信号機を1つに絞る
        trafficlight_label_list=[]
        trafficlight_confidence_list=[]
        trafficlight_bbox_list=[]
        for label, confidence, bbox in detections: 
            if(label=="Traffic light"):
                trafficlight_label_list.append(label)
                trafficlight_confidence_list.append(confidence)
                trafficlight_bbox_list.append(bbox)
        
        target_index = trafficlight_confidence_list.index(max(trafficlight_confidence_list))  # 検知した信号機のうち、1番信頼度の高いものを対象にし、そのindexを取ってくる
        target_bbox =  trafficlight_bbox_list[target_index]  # 取ってきたindexを基に、1版信頼度の高い信号機の座標を保存       
        xmin, ymin, xmax, ymax = darknet.bbox2points(target_bbox)
        xmin = xmin-1
        ymin = ymin-1
        xmax = xmax-1
        ymax = ymax-1
        print("xmin:{} ymin:{}, xmax:{}, ymax:{}".format(xmin, ymin, xmax, ymax), flush=True)
        # クロッピングする際、座標が負の値だとエラーになるので該当する変数は0にする
        if(xmin<0):
            xmin = 0
        elif(ymin<0):
            ymin = 0
        elif(xmax<0):
            xmax = 0
        elif(ymax<0):
            ymax = 0
            
        cropped_image = image[ymin:ymax, xmin:xmax]  # bboxの枠線が映らないようにちょっと狭めに切り取る
        
        #import numpy as np
        # 1.HSVに変換して白黒でマスク(2値化)
        hsv_cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
        # 信号機の目:白,それ以外:黒 でマスクする 
        # 彩度(s)・明度(v)で絞り込む. 信号機の目は大体鮮やかなのでそのようになるように指定
        lower = np.array([0, 200,100])           # 抽出する色の下限(h,s,v)
        upper = np.array([150, 255 , 255])        # 抽出する色の上限(h,s,v)
        mask_traffic_light = cv2.inRange(hsv_cropped_image, lower, upper) # inRangeで元画像を２値化
        
        # 1の結果を基に、信号機の目部分の色はそのままに、それ以外を黒でマスク
        target = cv2.bitwise_and(hsv_cropped_image,hsv_cropped_image, mask=mask_traffic_light)
        
        # 信号機の目以外は黒色(H=0)でマスクしている為、Hの平均値を出しても小さい値にしかならないので意味がない
        # → なのでHのminとmaxを出す(min要らないけど)
        h_min = target.T[0].flatten().min()
        h_max = target.T[0].flatten().max()
        #print("h_min: %s" % (str(h_min)))
        #print("h_max: %s" % (str(h_max)))
        
        # Hの最大値で判別する
        if(0 <= h_max <= 30):
            re = -1
            print("Traffic_light : red", flush=True)
        elif(30<h_max and h_max<=150):  # 緑と青は取り敢えず一緒にした
            re = 0
            print("Traffic_light : green", flush=True)
        else:
            re = -1
            print("Traffic_light : none", flush=True)
    else:
        now = datetime.datetime.now().isoformat(sep=' ', timespec='milliseconds')
        #filename = '/home/jetbot/g031r066/darknet/bad_snapshot/' + str(now) + '.jpg'
        #cv2.imwrite(filename, snapshot)  # 信号機を検知できなかったときのスナップショットを保存
    
    clear_output(True)
    queue_od.put(re)

In [9]:
import threading
from IPython.display import clear_output

def execute(change):
    snapshot = change['new']  # 映像読み込み type(snapshot) = numpy.ndarray

    # ライントレースと物体検知のスレッドをそれぞれ作成
    thread_rf = threading.Thread(target = road_following, args=(snapshot,))
    thread_od = threading.Thread(target = object_detection, args=(snapshot,))
    
    # カクカクになるけどちゃんと動いてくれるのでは?
    robot.left_motor.value = 0.0
    robot.right_motor.value = 0.0
    
    # 各スレッド実行開始
    thread_rf.start()
    thread_od.start()

    # 各スレッド内の処理が終わるまで待つ
    thread_rf.join()
    thread_od.join()
    
    # 各スレッド内での結果を受け取る
    left_motor_rf = queue_rf.get()   # ライントレース実行時における右タイヤの制御値
    right_motor_rf = queue_rf.get()  # ライントレース実行時における左タイヤの制御値
    image = queue_od.get()  # 物体検知実行時のbboxが描画された画像
    re = queue_od.get()  # 物体検知実行時の返り値
    
    if(re == -1):  # 赤信号時
        # 一時停止
        robot.left_motor.value = 0.0
        robot.right_motor.value = 0.0
        time.sleep(0.1)
    else:  # 赤信号以外
        robot.left_motor.value = left_motor_rf
        robot.right_motor.value = right_motor_rf

    # 出力されているカメラ映像の更新
    #image_widget.value = bgr8_to_jpeg(snapshot)
    bbox_widget.value = bgr8_to_jpeg(image)

In [10]:
from ipywidgets import HBox, VBox

#display(HBox([image_widget,bbox_widget]))
display(HBox([bbox_widget, VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider])]))

In [11]:
execute({'new': camera.value})
camera.observe(execute, names='value')


Objects:
Traffic sign: 74.25%    (left_x: 394   top_y:  378   width:   43   height:  64)
FPS: 4
Predicted in 0.2286677360534668 seconds


In [12]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
camera.stop()
robot.stop()

In [20]:
camera.stop()
robot.stop()

In [16]:
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0

In [ ]:
camera.restart()